In [1]:
import requests, json, os, time
from dotenv import load_dotenv, find_dotenv

# 加载环境变量
_ = load_dotenv(find_dotenv())

APPID  = os.getenv("APPID")
SECRET = os.getenv("APPSecret")

In [2]:
APPID,SECRET

('wx316a3ce78ed853a3', '176512388b7eb8213f63d18c0dae1ce0')

In [6]:
def get_access_token():
    url = "https://api.weixin.qq.com/cgi-bin/token"
    params = {"grant_type": "client_credential", "appid": APPID, "secret": SECRET}
    r = requests.get(url, params=params, timeout=10).json()
    if "access_token" not in r:          # 关键防护
        raise RuntimeError(f"WX error → {r}")   # 把 errcode/errmsg 打出来
    return r["access_token"]


In [8]:
# 测试access_token能否正常获取
get_access_token()

'92_sFTOzfVIYcmSySggTonqBeUDGJCENJsmPU9gmzJUGSdrQDVbzb7nePZLa8LgcSHshFF4Z-BYs0TztTd5TwgO2mfvDLiNDPI0wO2NU6tXSwYPwEMGYuCwo3bMXskOUHcAHAIOB'

In [11]:
def _parse_one(item):
    meta = item.get("content") or {}
    ts = meta.get("create_time") or item.get("update_time")
    ts_fmt = (
        time.strftime("%Y-%m-%d %H:%M", time.localtime(ts))
        if ts else "N/A"
    )
    for news in meta.get("news_item", []):
        yield {
            "title": news.get("title", ""),
            "url":   news.get("url", ""),
            "digest": news.get("digest", ""),
            "created": ts_fmt
        }

def list_published(offset=0, count=20, no_content=0):
    ak = get_access_token()
    r = requests.post(
        f"https://api.weixin.qq.com/cgi-bin/freepublish/batchget?access_token={ak}",
        json={"offset": offset, "count": count, "no_content": no_content},
        timeout=10
    ).json()

    if "item" not in r:
        raise RuntimeError(f"WX error → {r}")   # 继续沿用你的兜底

    articles = []
    for item in r["item"]:
        articles.extend(_parse_one(item))
    return articles


In [12]:
if __name__ == "__main__":
    for art in list_published():
        print(art["created"], art["title"], art["url"])

2025-02-05 12:17 deepseek æ¨¡åæ¬å°é¨ç½²æå http://mp.weixin.qq.com/s?__biz=Mzk2NDQ2ODU0OQ==&mid=2247483696&idx=1&sn=0b23bde1d6b2f9c5a9f72bd6725b0b37&chksm=c4724b1af305c20c6ce8d01d1b71cf926a1666c8712e561f2e88e6e15fe029f7c602666dbfae#rd
